<a href="https://colab.research.google.com/github/rafaelrlima/smarkio_test_data_science/blob/main/smarkio_analise_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [13]:
column_names = ["letra", "artista"]

xls = '/content/drive/MyDrive/Colab Notebooks/teste_smarkio_lbs.xls'

df = pd.read_excel(xls, sheet_name='NLP',usecols=column_names)

In [14]:
df.head

<bound method NDFrame.head of                                                  letra  artista
0    Jay-z Uh-uh-uh You ready b? Let's go get 'em. ...  Beyoncé
1    Your challengers are a young group from Housto...  Beyoncé
2    Dum-da-de-da Do, do, do, do, do, do (Coming do...  Beyoncé
3    If I ain't got nothing I got you If I ain't go...  Beyoncé
4    Six inch heels She walked in the club like nob...  Beyoncé
..                                                 ...      ...
513  Yeah yeah Yeah yeah  I ain't trying to think a...  Rihanna
514  You the one that I dream about all day You the...  Rihanna
515  No, no, no You don't love me and I know now No...  Rihanna
516  You should be mine Oh baby, oh baby, oh baby, ...  Rihanna
517  [Rihanna] I remember when the world was just m...  Rihanna

[518 rows x 2 columns]>